In [1]:
import os
import re
import json

In [ ]:
import os
import re
import json

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

---

In [68]:
translate_folder = "../data_translate/"
files = os.listdir(translate_folder)

In [69]:
data_list = []
for file in files:
    with open(f"{translate_folder}{file}", "r", encoding="utf-8") as f:
        data_list.append(json.load(f))

In [71]:
df = pd.DataFrame(data_list, index=range(len(data_list)))
df

,article,highlights,vi
0,For years they have been a source of joy for y...,More youngsters end up in hospital after tramp...,Nhiều thanh niên phải nhập viện sau khi nhiếp ...
1,Once again: a hidden camera has caught New Yor...,Woman walks around New York in yoga pants with...,Người phụ nữ đi dạo quanh New York trong chiếc...
2,I thought it would be a good chance for Roy Ho...,James Milner did not move the ball quickly eno...,James Milner di chuyển bóng không đủ nhanh tro...
3,3D reconstructions of Richard III's spine show...,Shakespeare described Richard III as a 'poison...,Shakespeare mô tả Richard III là 'con lưng gù ...
4,This is the moment a brave buffalo charged thr...,The buffalo and its young became separated fro...,Con trâu và đàn con bị ba sư tử đói ở Tanzania...
...,...,...,...
1017,A new Red Bull advert which makes light of the...,Advert shows Titanic captain dismissing crate ...,Quảng cáo cho thấy thuyền trưởng Titanic đang ...
1018,By . Laura Silver for MailOnline . Everyone lo...,Two-year-old Miles is intent on getting forty ...,Cậu bé Miles hai tuổi đang có ý định nhận được...
1019,By . Associated Press Reporter and Daily Mail ...,Tampa socialite who tipped off FBI about Petra...,"Trang xã hội Tampa, người đã tiết lộ cho FBI v..."
1020,Devastated British tycoon Alan Bond yesterday ...,"Diana Bliss, 57, had stood by husband's side a...","Diana Bliss, 57 tuổi, đã sát cánh bên chồng sa..."


---

In [8]:
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\vannt\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
tokenizer_en2vi = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi-v2", src_lang="en_XX")
model_en2vi = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi-v2")

  0%|          | 0/615 [02:26<?, ?it/s]


In [10]:
def translate_en2vi(en_texts: str) -> str:
    input_ids = tokenizer_en2vi(en_texts, padding=True, return_tensors="pt")
    output_ids = model_en2vi.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [11]:
# The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
en_texts = ["More youngsters end up in hospital after trampolining than skateboarding .Children suffer worst injuries when bouncing at same time as an adult .Chance of injury increases if there is more than one child on trampoline .Doctors in America have called for them to be banned due to casualties ."]
print(translate_en2vi(en_texts))

['Nhiều trẻ em kết thúc trong bệnh viện sau khi trampolining hơn trượt ván. Trẻ em bị thương nặng nhất khi nảy cùng lúc với người lớn. Thay đổi chấn thương tăng lên nếu có nhiều hơn một đứa trẻ trên trampoline. Các bác sĩ ở Mỹ đã kêu gọi họ bị cấm do thương vong.']


---

In [52]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [53]:
# Tạo DataCollator
data_collator = DataCollatorForSeq2Seq(tokenizer_en2vi)

# Chuẩn bị dữ liệu cho Seq2SeqTrainer
train_data = list(zip(train_df["highlights"].tolist(), train_df["vi"].tolist()))
test_data = list(zip(test_df["highlights"].tolist(), test_df["vi"].tolist()))

In [25]:
# Chuẩn bị dữ liệu cho Seq2SeqTrainer
train_data = {
    "input_ids": tokenizer_en2vi(train_df["highlights"].tolist(), return_tensors="pt", padding=True).input_ids,
    "labels": tokenizer_en2vi(train_df["vi"].tolist(), return_tensors="pt", padding=True).input_ids,
}
test_data = {
    "input_ids": tokenizer_en2vi(test_df["highlights"].tolist(), return_tensors="pt", padding=True).input_ids,
    "labels": tokenizer_en2vi(test_df["vi"].tolist(), return_tensors="pt", padding=True).input_ids,
}

In [26]:
# Chuẩn bị đối số cho fine-tuning
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=2,
    num_train_epochs=3,
    save_steps=1000,
    logging_steps=500,
    learning_rate=5e-5,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_dir="./logs",
)

In [27]:
# Fine-tuning mô hình
trainer = Seq2SeqTrainer(
    model=model_en2vi,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
)


In [28]:
trainer.train()

KeyError: 0

In [ ]:
# Fine-tune the model
trainer.train()

# Evaluate the fine-tuned model on the test set
results = trainer.evaluate(test_dataset)

# Save the fine-tuned model
trainer.save_model("./fine-tuned-model")

In [31]:
import os
import json
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data.iloc[idx]["highlights"]
        target_text = self.data.iloc[idx]["vi"]
        encoding = self.tokenizer(source_text, return_tensors="pt", padding=True, truncation=True)
        labels = self.tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)
        labels["input_ids"] = labels["input_ids"].squeeze()
        labels["attention_mask"] = labels["attention_mask"].squeeze()
        return {k: v.squeeze() for k, v in encoding.items(), "labels": labels["input_ids"]}

# Đọc dữ liệu từ các tệp trong thư mục
translate_folder = "../data_translate/"
files = os.listdir(translate_folder)
data_list = []
for file in files:
    with open(f"{translate_folder}{file}", "r", encoding="utf-8") as f:
        data_list.append(json.load(f))

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data_list, index=range(len(data_list)))

# Tạo tokenizer và mô hình
tokenizer_en2vi = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi-v2", src_lang="en_XX")
model_en2vi = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi-v2")

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Tạo custom dataset
train_dataset = CustomDataset(train_df, tokenizer_en2vi)
test_dataset = CustomDataset(test_df, tokenizer_en2vi)

# Chuẩn bị đối số cho fine-tuning
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=2,
    num_train_epochs=3,
    save_steps=1000,
    logging_steps=500,
    learning_rate=5e-5,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_dir="./logs",
)

# Fine-tuning mô hình
trainer = Seq2SeqTrainer(
    model=model_en2vi,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer_en2vi),
)

trainer.train()

SyntaxError: invalid syntax (2333498997.py, line 23)

---

---

In [68]:
import os
import re
import json
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

In [69]:
translate_folder = "../data_translate/"
files = os.listdir(translate_folder)

In [70]:
data_list = []
for file in files:
    with open(f"{translate_folder}{file}", "r", encoding="utf-8") as f:
        data_list.append(json.load(f))

In [71]:
df = pd.DataFrame(data_list, index=range(len(data_list)))
df

,article,highlights,vi
0,For years they have been a source of joy for y...,More youngsters end up in hospital after tramp...,Nhiều thanh niên phải nhập viện sau khi nhiếp ...
1,Once again: a hidden camera has caught New Yor...,Woman walks around New York in yoga pants with...,Người phụ nữ đi dạo quanh New York trong chiếc...
2,I thought it would be a good chance for Roy Ho...,James Milner did not move the ball quickly eno...,James Milner di chuyển bóng không đủ nhanh tro...
3,3D reconstructions of Richard III's spine show...,Shakespeare described Richard III as a 'poison...,Shakespeare mô tả Richard III là 'con lưng gù ...
4,This is the moment a brave buffalo charged thr...,The buffalo and its young became separated fro...,Con trâu và đàn con bị ba sư tử đói ở Tanzania...
...,...,...,...
1017,A new Red Bull advert which makes light of the...,Advert shows Titanic captain dismissing crate ...,Quảng cáo cho thấy thuyền trưởng Titanic đang ...
1018,By . Laura Silver for MailOnline . Everyone lo...,Two-year-old Miles is intent on getting forty ...,Cậu bé Miles hai tuổi đang có ý định nhận được...
1019,By . Associated Press Reporter and Daily Mail ...,Tampa socialite who tipped off FBI about Petra...,"Trang xã hội Tampa, người đã tiết lộ cho FBI v..."
1020,Devastated British tycoon Alan Bond yesterday ...,"Diana Bliss, 57, had stood by husband's side a...","Diana Bliss, 57 tuổi, đã sát cánh bên chồng sa..."


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     1022 non-null   object
 1   highlights  1022 non-null   object
 2   vi          1022 non-null   object
dtypes: object(3)
memory usage: 24.1+ KB


In [79]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

In [80]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
model = TFAutoModelForSeq2SeqLM.from_pretrained("VietAI/envit5-translation")

tf_model.h5:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

c:\Users\vannt\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vannt\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\vannt\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\initializers\initializers.py:120: Us

In [74]:
inputs = df["highlights"].values.tolist()
targets =df["vi"].values.tolist()
# data_dict = {"inputs": inputs, "targets": targets}

In [75]:
# Tạo Dataset
dataset = Dataset.from_dict({
    "inputs": inputs,
    "targets": targets
})

# Tạo DatasetDict
dataset_dict = DatasetDict({
    "train": dataset
})

In [76]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 1022
    })
})

In [77]:
max_input_length = 200
max_target_length = 200
                         
def preprocess_function(examples):
    model_inputs = tokenizer( examples['inputs'],
    max_length=max_input_length,
    # return_tensors="pt",
    # padding=True,
    truncation=True
    )

    labels = tokenizer(
        text_target=examples["targets"],
        max_length=max_target_length,
        # padding=True,
        truncation=True
    )


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [78]:
tokenizer_data = dataset_dict.map(preprocess_function, batched = True, remove_columns = ['inputs', 'targets'])

Map:   0%|          | 0/1022 [00:00<?, ? examples/s]

In [58]:
tokenizer_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1022
    })
})

In [63]:
count_different_length = 0
max_target_length = 200

for i in range(1022):
    label_length = len(tokenizer_data['train'][i]['labels'])  # Lưu ý: labels là list trong tokenizer_data
    if label_length != max_target_length:
        count_different_length += 1

print("Số lượng chuỗi có chiều dài khác 200:", count_different_length)


Số lượng chuỗi có chiều dài khác 200: 22


In [65]:
max_target_length = 200

for i in range(len(tokenizer_data['train'])):
    label_length = len(tokenizer_data['train'][i]['labels'])  # Lưu ý: labels là list trong tokenizer_data
    if label_length != max_target_length:
        print(f"Index {i}: {label_length}")


Index 1000: 116
Index 1001: 116
Index 1002: 116
Index 1003: 116
Index 1004: 116
Index 1005: 116
Index 1006: 116
Index 1007: 116
Index 1008: 116
Index 1009: 116
Index 1010: 116
Index 1011: 116
Index 1012: 116
Index 1013: 116
Index 1014: 116
Index 1015: 116
Index 1016: 116
Index 1017: 116
Index 1018: 116
Index 1019: 116
Index 1020: 116
Index 1021: 116


In [81]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [82]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [83]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [84]:
train_dataset = model.prepare_tf_dataset(
    tokenizer_data["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [85]:
from transformers import AdamWeightDecay

In [86]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [87]:
model.fit(train_dataset, epochs=1)

 1/63 [..............................] - ETA: 2:20:09 - loss: 1.7770

---

In [60]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_fine_tuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    num_train_epochs=3,
)

# Define the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_data["train"],
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

  0%|          | 0/768 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [12]:
tokenizer_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1022
    })
})

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
tf_train_dataset = tokenizer_data['train'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [21]:
from transformers import create_optimizer

In [22]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenizer_data) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [23]:
import tensorflow as tf

In [24]:
model.compile(optimizer=optimizer)

TypeError: compile() got an unexpected keyword argument 'optimizer'

In [ ]:
model.fit(x=tf_train_dataset, epochs=5)

In [98]:
# Chuẩn bị đối số cho fine-tuning
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=2,
    num_train_epochs=3,
    save_steps=1000,
    logging_steps=500,
    learning_rate=5e-5,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_dir="./logs",
)

In [101]:
# Fine-tuning mô hình
trainer = Seq2SeqTrainer(
    model=model_en2vi,
    args=training_args,
    train_dataset=dataset_dict["train"],
    data_collator=DataCollatorForSeq2Seq(tokenizer_data),
)

In [102]:
trainer.train()

  0%|          | 0/768 [00:00<?, ?it/s]

IndexError: Invalid key: 453 is out of bounds for size 0